In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer


In [2]:
path = "./yellow_tripdata_2023-01.parquet"
path2 = "./yellow_tripdata_2023-02.parquet"

In [3]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    #df.head()
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    # filter
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    
    df[categorical] = df[categorical].astype(str) #int -> str
    
    return df


In [ ]:
#df = pd.read_parquet(path)
df = read_dataframe(path).sample(10000, random_state=1)
df_val = read_dataframe(path2).sample(10000, random_state=1)


In [6]:
# 1.
# print(len(df.columns))

In [7]:
# 2. duration 컬럼의 표준편차 구하기
# df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
# df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

# df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
# df['duration'] = df['duration'].dt.total_seconds() / 60

# print(f"Duration 표준편차: {df['duration'].std():.2f} 분") # mean median ...

In [8]:
#3. Fraction of records
# 전체 데이터 중에 필터링된 데이터의 비율
# total = len(df)
# cond_trips = len(df[(df['duration'] >= 1) & (df['duration'] <= 60)])
# fraction_normal = cond_trips / total
# fraction_normal


In [9]:
#4. one hot encoding - 카테고리형 데이터를 모델이 이해할 수 있는 숫자로
# df['PULocationID'] = df['PULocationID'].astype(str)
# df['DOLocationID'] = df['DOLocationID'].astype(str)
# # label encoding을 막기 위해 string으로 변환
pu = ['PULocationID']
do = ['DOLocationID']

dv = DictVectorizer()
some_dict = df[pu + do].to_dict(orient='records')  # 각 row를 하나의 딕셔너리로 


In [10]:

# feature matrix : 모델의 입력 데이터 행렬
X_train = dv.fit_transform(some_dict)
# 카테고리형 데이터의 개수가 feature의 개수 즉, feature matrix의 차원이 된다.
X_train.shape # get rows and columns

#print(X_train[:15, :10].toarray())


(3009173, 515)

Label Encoding vs One-hot Encoding 비교:
1. Label Encoding:

```python
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
locations = ['A', 'B', 'C', 'A', 'B']
encoded = le.fit_transform(locations)  # [0, 1, 2, 0, 1]
```


2. One-hot Encoding (현재 방법):

```python
# 결과: 각 카테고리마다 별도 컬럼
# A: [1, 0, 0, 1, 0]
# B: [0, 1, 0, 0, 1] 
# C: [0, 0, 1, 0, 0]
```
언제 어떤 방법을 사용하나요?
Label Encoding 사용:
순서가 있는 카테고리 (예: 'Low', 'Medium', 'High')
트리 기반 모델 (RandomForest, XGBoost)
메모리 효율성이 중요할 때
One-hot Encoding 사용 (현재 방법):
순서가 없는 카테고리 (위치 ID처럼)
선형 모델 (Linear Regression, Logistic Regression)
카테고리 간 동등한 관계일 때

In [11]:
# 5. Model training
# categorical = ['PULocationID', 'DOLocationID']
# numerical = ['duration']

# train_dicts = df[categorical + numerical].to_dict(orient='records')
# x_train = dv.fit_transform(train_dicts)

In [12]:
# val_dicts = df_val[categorical + numerical].to_dict(orient='records')
# x_val = dv.transform(val_dicts)

In [13]:
#예측하고자 하는 목표값(여기서는 'duration' 컬럼)을 y_train에 저장합니다.
# target = 'duration'
# y_train = df[target].values
# y_val = df_val[target].values
